In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# get data files
names = ["class", "message"]
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"
train_file = pd.read_csv(train_file_path, sep='\t', names=names)
test_file = pd.read_csv(test_file_path, sep='\t', names=names)

In [ ]:
test_file = pd.read_csv(test_file_path, sep='\t', names=names)

train_message = train_file["message"].values.tolist()
train_label = np.array([0 if x=="ham" else 1 for x in train_file['class'].values.tolist()])
test_message = test_file["message"].values.tolist()
test_label = np.array([0 if x=="ham" else 1 for x in test_file['class'].values.tolist()])


vocabulary_dict = {}
for messgae in train_message:
  for vocabulary in messgae.split():
    if vocabulary not in vocabulary_dict:
      vocabulary_dict[vocabulary] = 1
    else:
      vocabulary_dict[vocabulary] += 1


In [ ]:


VOCAB_SIZE = len(vocabulary_dict)
MAX_LENGTH = len(max(train_message, key=lambda p: len(p.split())).split())



In [ ]:
# Encode train messages
from sklearn.feature_extraction.text import CountVectorizer

# Initialize CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform on training messages
X_train_encoded = vectorizer.fit_transform(train_message)
# Transform test messages
X_test_encoded = vectorizer.transform(test_message)

# Convert to arrays and pad sequences
X_train_padded = X_train_encoded.toarray()
X_test_padded = X_test_encoded.toarray()


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import FunctionTransformer

# Define a pipeline
model = Pipeline([
    ('embedding', FunctionTransformer(lambda x: x)),
    ('flatten', FunctionTransformer(lambda x: x.reshape((x.shape[0], -1)))),
    ('dense', LogisticRegression())
])

# Train the model
model.fit(X_train_padded, train_label)

# Evaluate the model
train_accuracy = accuracy_score(train_label, model.predict(X_train_padded))
test_accuracy = accuracy_score(test_label, model.predict(X_test_padded))

print("Training Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

def predict_message(pred_text):
    class_dict = {0: "ham", 1: "spam"}

    # Transform the text using the CountVectorizer
    pred_text_encoded = vectorizer.transform([pred_text])

    # Predict the class
    prediction_prob = model.predict_proba(pred_text_encoded)[0][1]
    prediction_label = class_dict[model.predict(pred_text_encoded)[0]]

    return [prediction_prob, prediction_label]


In [ ]:

def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()



In [ ]:
train_message = train_file["message"].values.tolist()
train_label = np.array([0 if x=="ham" else 1 for x in train_file['class'].values.tolist()])
test_message = test_file["message"].values.tolist()
test_label = np.array([0 if x=="ham" else 1 for x in test_file['class'].values.tolist()])

from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TfidfVectorizer
vectorizer = TfidfVectorizer()

# Fit and transform on training messages
X_train_encoded = vectorizer.fit_transform(train_message)
# Transform test messages
X_test_encoded = vectorizer.transform(test_message)

# Convert to arrays and pad sequences
X_train_padded = X_train_encoded.toarray()
X_test_padded = X_test_encoded.toarray()

from sklearn.svm import SVC

# Define a pipeline with SVM classifier
# Define a pipeline with SVC classifier
model = Pipeline([
    ('embedding', FunctionTransformer(lambda x: x)),
    ('svm', SVC(kernel='linear', probability=True))
])

# Train the model
model.fit(X_train_encoded, train_label)

# Evaluate the model
train_accuracy = accuracy_score(train_label, model.predict(X_train_encoded))
test_accuracy = accuracy_score(test_label, model.predict(X_test_encoded))

print("Training Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)


In [ ]:
# Function to predict messages based on the trained model
def predict_message(pred_text):
    class_dict = {0: "ham", 1: "spam"}

    # Transform the text using the TfidfVectorizer
    pred_text_encoded = vectorizer.transform([pred_text])

    # Predict the class
    prediction_prob = model.predict_proba(pred_text_encoded)[0][1]
    prediction_label = class_dict[model.predict(pred_text_encoded)[0]]

    return [prediction_prob, prediction_label]

# Test predictions
def test_predictions():
    test_messages = ["how are you doing today",
                    "sale today! to stop texts call 98912460324",
                    "i dont want to go. can we try it a different day? available sat",
                    "our new mobile video service is live. just install on your phone to start watching.",
                    "you have won £1000 cash! call to claim your prize.",
                    "i'll bring it tomorrow. don't forget the milk.",
                    "wow, is your arm alright. that happened to me one time too"
                    ]
    test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]

    passed = True

    for msg, ans in zip(test_messages, test_answers):
        prediction = predict_message(msg)
        print("Message:", msg)
        print("Prediction:", prediction[1])
        print("Expected:", ans)
        print("--------------------------")
        if prediction[1] != ans:
            passed = False

    if passed:
        print("You passed the challenge. Great job!")
    else:
        print("You haven't passed yet. Keep trying.")

# Test the predictions
test_predictions()
